In [13]:
#Check if the folder exists, if not create a new path, improve code stability
import os
import numpy as np
import pandas as pd
import zipfile

from urllib.request import urlretrieve

#Go back one level from notebook to data 
output_relative_dir = '../data/'

if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
for target_dir in ('curated','outer'):
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

In [14]:
url = "https://www.abs.gov.au/census/find-census-data/datapacks/download/2021_GCP_SA2_for_AUS_short-header.zip"#year-month.parquet
output_dir = "../data/outer/census.zip"
# download
urlretrieve(url, output_dir) 
print("complete")

complete


In [15]:
files = zipfile.ZipFile('../data/outer/census.zip','r')

for file in files.namelist():
    files.extract(file, f"../data/outer/census")

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

In [17]:
G04= spark.read.option("header",True).csv('../data/outer/census/2021 Census GCP Statistical Area 2 for AUS/2021Census_G17A_AUST_SA2.csv') #read the parquet 
G = G04.na.drop(subset=G04.columns)
print(G04.count(),G.count())

2472 2472


In [18]:
url = "http://github.com/matthewproctor/australianpostcodes/zipball/master"#year-month.parquet
output_dir = "../data/outer/total_pto2.zip"
# download
urlretrieve(url, output_dir) 
print("complete")


complete


In [19]:
files = zipfile.ZipFile('../data/outer/total_pto2.zip','r')
for file in files.namelist():
    files.extract(file, f"../data/outer/total_pto2")

In [20]:
post_sa2= spark.read.option("header",True).csv('../data/outer/total_pto2/matthewproctor-australianpostcodes-6f8a994/australian_postcodes.csv') #read the parquet 
post_sa2

id,postcode,locality,state,long,lat,dc,type,status,sa3,sa3name,sa4,sa4name,region,Lat_precise,Long_precise,SA1_MAINCODE_2011,SA1_MAINCODE_2016,SA2_MAINCODE_2016,SA2_NAME_2016,SA3_CODE_2016,SA3_NAME_2016,SA4_CODE_2016,SA4_NAME_2016,RA_2011,RA_2016,MMM_2015,MMM_2019,ced,altitude,chargezone,phn_code,phn_name,lgaregion,electorate,electoraterating
230,0200,ANU,ACT,149.119,-35.2777,null,null,null,null,null,null,null,R1,-35.2777,149.119,80105104901,80105104901,801051049,Acton,80105,North Canberra,801,Australian Capita...,1,1,1,1,null,null,N2,null,null,null,Durack,null
21820,0200,Australian Nation...,ACT,149.1189,-35.2777,null,null,Added 19-Jan-2020,null,null,null,null,R1,-35.2776999,149.118527,80105104901,80105104901,801051049,Acton,80105,North Canberra,801,Australian Capita...,1,1,1,1,null,null,N2,null,null,null,Durack,null
232,0800,DARWIN,NT,130.83668,-12.458684,null,null,Updated 6-Feb-2020,70101,Darwin City,701,Darwin,R1,-12.3932794,130.7766611,70101100203,70101100218,701011002,Darwin City,70101,Darwin City,701,Darwin,3,3,2,2,null,null,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan
24049,0800,DARWIN CITY,NT,130.83668,-12.458684,null,null,Updated 6-Feb-2020,70101,Darwin City,701,Darwin,R1,-12.3932794,130.7766611,70101100203,70101100218,701011002,Darwin City,70101,Darwin City,701,Darwin,3,3,2,2,null,null,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan
233,0801,DARWIN,NT,130.83668,-12.458684,null,null,Updated 25-Mar-20...,70101,Darwin City,701,Darwin,R1,-12.4634403,130.8456418,70101100208,70101100208,701011002,Darwin City,70101,Darwin City,701,Darwin,3,3,2,2,null,null,NT1,PHN701,null,null,Lingiari,Rural
234,0804,PARAP,NT,130.873315,-12.428017,null,null,Updated 25-Mar-20...,70102,Darwin Suburbs,701,Darwin,R1,-12.4324801,130.8462536,70101100704,70101100704,701011007,Parap,70101,Darwin City,701,Darwin,3,3,2,2,null,null,NT1,PHN701,null,null,Durack,null
235,0810,ALAWA,NT,130.866242,-12.381806,null,null,Updated 6-Feb-2020,70102,Darwin Suburbs,701,Darwin,R1,-12.38,130.873,70102101001,70102101001,701021010,Alawa,70102,Darwin Suburbs,701,Darwin,3,3,2,2,null,9.22142887115479,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan
236,0810,BRINKIN,NT,130.866242,-12.381806,null,null,Updated 6-Feb-2020,70102,Darwin Suburbs,701,Darwin,R1,-12.37,130.867,70102101301,70102101301,701021013,Brinkin - Nakara,70102,Darwin Suburbs,701,Darwin,3,3,2,2,null,9.22142887115479,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan
237,0810,CASUARINA,NT,130.866242,-12.381806,null,null,Updated 6-Feb-2020,70102,Darwin Suburbs,701,Darwin,R1,-12.373333,130.881667,70102101307,70102101307,701021013,Brinkin - Nakara,70102,Darwin Suburbs,701,Darwin,3,3,2,2,null,9.22142887115479,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan
19972,0810,COCONUT GROVE,NT,130.8502369,-12.39614166,Northern Territor...,Delivery Area,Updated 6-Feb-2020,70102,Darwin Suburbs,701,Darwin,R1,-12.3989,130.852,70102101604,70102101604,701021016,Coconut Grove,70102,Darwin Suburbs,701,Darwin,3,3,2,2,null,9.22142887115479,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan


In [21]:
url = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip" 
#year-month.parquet
output_dir = "../data/outer/2021sa2_shapefile.zip"
# download
urlretrieve(url, output_dir) 
print("complete")

complete


In [22]:
files = zipfile.ZipFile('../data/outer/2021sa2_shapefile.zip','r')
for file in files.namelist():
    files.extract(file, f"../data/outer/2021sa2_shapefile")

In [23]:
import pandas as pd
from dbfread import DBF
path = r'../data/outer/2021sa2_shapefile/SA2_2021_AUST_GDA2020.dbf' # 文件目录
table = DBF(path)
df = pd.DataFrame(iter(table))
df.to_csv("../data/outer/2021sa2_shapefile/SA2_2021_AUST_GDA2020.csv",index_label="FID")

In [24]:
SA2_2021_shapefile= spark.read.option("header",True).csv('../data/outer/2021sa2_shapefile/SA2_2021_AUST_GDA2020.csv') #read the parquet 
SA2_2021_shapefile = SA2_2021_shapefile.na.drop()
SA2_2021_shapefile

FID,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21
0,101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.dat...
1,101021008,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.dat...
2,101021009,Queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.762,http://linked.dat...
3,101021010,Queanbeyan - East,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.dat...
4,101021012,Queanbeyan West -...,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.dat...
5,101021610,Googong,1,New,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9164,http://linked.dat...
6,101021611,Queanbeyan Surrounds,1,New,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3047.7057,http://linked.dat...
7,101031013,Bombala,0,No change,10103,Snowy Mountains,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3990.144,http://linked.dat...
8,101031014,Cooma,0,No change,10103,Snowy Mountains,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,103.6371,http://linked.dat...
9,101031015,Cooma Surrounds,3,Name change,10103,Snowy Mountains,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6251.0158,http://linked.dat...


In [25]:
url = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/correspondences/CG_SA2_2016_SA2_2021.csv"
output_dir = "../data/outer/correspondences.csv"
# download
urlretrieve(url, output_dir) 
print("complete")

complete


In [28]:
correspondences= spark.read.option("header",True).csv('../data/outer/correspondences.csv') #read the parquet 
correspondences = correspondences.na.drop()
correspondences

SA2_MAINCODE_2016,SA2_NAME_2016,SA2_CODE_2021,SA2_NAME_2021,RATIO_FROM_TO,INDIV_TO_REGION_QLTY_INDICATOR,OVERALL_QUALITY_INDICATOR,BMOS_NULL_FLAG
101021007,Braidwood,101021007,Braidwood,1,Good,Good,0
101021008,Karabar,101021008,Karabar,1,Good,Good,0
101021009,Queanbeyan,101021009,Queanbeyan,1,Good,Good,0
101021010,Queanbeyan - East,101021010,Queanbeyan - East,1,Good,Good,0
101021011,Queanbeyan Region,101021610,Googong,0.0967097,Poor,Good,0
101021011,Queanbeyan Region,101021611,Queanbeyan Surrounds,0.9032903,Good,Good,0
101021012,Queanbeyan West -...,101021012,Queanbeyan West -...,1,Good,Good,0
101031013,Bombala,101031013,Bombala,1,Good,Good,0
101031014,Cooma,101031014,Cooma,1,Good,Good,0
101031015,Cooma Region,101031015,Cooma Surrounds,1,Good,Good,0
